In [1]:
import json
import pickle

import pandas as pd
import geopandas as gpd
import numpy as np
import networkx as nx
import osmnx as ox

import scipy.stats as stats
import matplotlib.pyplot as plt

ox.config(log_console=True)

num_bins = 36
threshold = 10

In [2]:
graphs = pickle.load(open("graphs.p", "rb"))

In [3]:
def reverse_bearing(x):
    return x + 180 if x < 180 else x - 180

def get_unweighted_bearings(G, threshold):
    # calculate edge bearings
    # threshold lets you discard streets < some length from the bearings analysis
    b = pd.Series([d['bearing'] for u, v, k, d in G.edges(keys=True, data=True) if d['length'] > threshold])
    return pd.concat([b, b.map(reverse_bearing)]).reset_index(drop='True')

In [4]:
def count_and_merge(n, bearings):
    # make twice as many bins as desired, then merge them in pairs
    # prevents bin-edge effects around common values like 0° and 90°
    n = n * 2
    bins = np.arange(n + 1) * 360 / n
    count, _ = np.histogram(bearings, bins=bins)
    
    # move the last bin to the front, so eg 0.01° and 359.99° will be binned together
    count = np.roll(count, 1)
    return count[::2] + count[1::2]

In [5]:
def calculate_orientation_entropy(data, n):
    bin_counts = count_and_merge(n, data)
    entropy = stats.entropy(bin_counts)
    return entropy

In [8]:
def circuity(G, edge_length_total):
    
    coords = np.array([[G.nodes[u]['y'], G.nodes[u]['x'], G.nodes[v]['y'], G.nodes[v]['x']] for u, v, k in G.edges(keys=True)])
    df_coords = pd.DataFrame(coords, columns=['u_y', 'u_x', 'v_y', 'v_x'])

    gc_distances = ox.distance.great_circle_vec(lat1=df_coords['u_y'],
                                                lng1=df_coords['u_x'],
                                                lat2=df_coords['v_y'],
                                                lng2=df_coords['v_x'])

    gc_distances = gc_distances.fillna(value=0)
    circuity_avg = edge_length_total / gc_distances.sum()
    return circuity_avg

In [10]:
cities = pd.read_csv("cities.csv")

In [11]:
%%time
results = {}

for i in np.arange(0, len(cities.city)):
    
    G = graphs[i]
    
    Gu = ox.get_undirected(G)
    lengths = pd.Series(nx.get_edge_attributes(Gu, 'length'))
    
    k_avg = 2 * len(Gu.edges()) / len(Gu.nodes())
    n = len(Gu.nodes())
    m = len(Gu.edges())
    length_median = lengths.median()
    length_mean = lengths.mean()
    
    # proportion of 4-way ints, dead-ends, and avg circuity
    prop_4way = list(Gu.graph['streets_per_node'].values()).count(4) / len(Gu.nodes())
    prop_deadend = list(Gu.graph['streets_per_node'].values()).count(1) / len(Gu.nodes())
    circuity_avg = circuity(Gu, lengths.sum())
    
    # calculate length entropy
    count, _ = np.histogram(lengths, num_bins)
    length_entropy = stats.entropy(count)
    count_log, _ = np.histogram(np.log(lengths+0.01), num_bins)
    length_entropy_log = stats.entropy(count_log)
    
    # calculate orientation entropy
    bearings = get_unweighted_bearings(ox.add_edge_bearings(Gu), threshold)
    orientation_entropy = calculate_orientation_entropy(bearings.dropna(), num_bins)
    
    city = cities.city[i]
    
    results[city] = {'k_avg'              : k_avg,
                     'n'                  : n,
                     'm'                  : m,
                     'prop_4way'          : prop_4way,
                     'prop_deadend'       : prop_deadend,
                     'circuity_avg'       : circuity_avg,
                     'length_median'      : length_median,
                     'length_mean'        : length_mean,
                     'length_entropy'     : length_entropy,
                     'length_entropy_log' : length_entropy_log,
                     'orientation_entropy': orientation_entropy}

CPU times: user 5min 21s, sys: 522 ms, total: 5min 22s
Wall time: 5min 22s


In [19]:
measures = pd.DataFrame(results).T.reset_index()
measures = measures.rename(columns={'index': "city"})

In [21]:
max_entropy = np.log(num_bins)
max_entropy

3.58351893845611

In [22]:
min_bins = 4 
perfect_grid = [1] * min_bins + [0] * (num_bins - min_bins)
perfect_grid_entropy = stats.entropy(perfect_grid)
perfect_grid_entropy

1.3862943611198906

In [24]:
def orientation_order(eta, max_ent=max_entropy, min_ent=perfect_grid_entropy):
    # normalize it as a value between perfect_grid_entropy and max_entropy
    # then square it to approx linearize orientation_order's relationship with the
    # share of total bins with equal non-zero probabilities
    return 1 - ((eta - min_ent) / (max_ent - min_ent)) ** 2

measures['orientation_order'] = measures['orientation_entropy'].map(orientation_order)

measures.head()

,city,k_avg,n,m,prop_4way,prop_deadend,circuity_avg,length_median,length_mean,length_entropy,length_entropy_log,orientation_entropy,orientation_order
0,New York City,3.293023,2150.0,3540.0,0.476744,0.079070,1.038945,18.6605,39.283738,1.682079,2.735562,3.385253,0.172327
1,Los Angeles,2.996134,1552.0,2325.0,0.382088,0.150129,1.078653,26.4980,51.805418,1.730214,3.088774,3.110549,0.384180
2,Chicago,3.073418,2765.0,4249.0,0.409042,0.150452,1.051320,19.2070,39.650949,1.351369,2.901023,2.421493,0.778028
3,Houston,3.031579,1425.0,2160.0,0.312281,0.095439,1.064071,38.3565,55.115096,1.779185,2.881514,3.184961,0.329880
4,Phoenix,2.720307,261.0,355.0,0.107280,0.149425,1.178540,62.9290,103.169411,2.413476,3.167428,2.435797,0.771851


In [25]:
measures['m'] = measures['m'].astype(int)
measures['n'] = measures['n'].astype(int)

In [26]:
measures.to_csv("measures_grid.csv")

In [27]:
with open("google.json", "r") as token:
    GOOGLE_KEY = json.load(token)["key"]

In [57]:
%%time
results = {}

for i in np.arange(0, len(cities)):
    
    G = ox.get_undirected(graphs[i])
    G = ox.add_node_elevations(G, api_key=GOOGLE_KEY)
    G = ox.add_edge_grades(G)
    
    e_grad = pd.Series(nx.get_edge_attributes(G, "grade_abs"))
    n_elev = pd.Series(nx.get_node_attributes(G, "elevation"))
    
    gradient_iqr = stats.iqr(e_grad)
    gradient_max = max(e_grad)
    gradient_min = min(e_grad)
    
    elevation_iqr = stats.iqr(n_elev)
    elevation_max = max(n_elev)
    elevation_min = min(n_elev)
    
    city = cities.city[i]
    
    results[city] = {'gradient_iqr'       : gradient_iqr,
                     'gradient_max'       : gradient_max,
                     'gradient_min'       : gradient_min,
                     'elevation_iqr'      : elevation_iqr,
                     'elevation_max'      : elevation_max,
                     'elevation_min'      : elevation_min}

CPU times: user 5min 38s, sys: 2.13 s, total: 5min 40s
Wall time: 1h 7min 7s


In [58]:
measures = pd.DataFrame(results).T.reset_index()
measures = measures.rename(columns={'index': "city"})

In [59]:
measures.to_csv("measures_elev.csv")

In [69]:
edges = gpd.read_file("edges_us.gpkg")
nodes = gpd.read_file("nodes_us.gpkg")

In [99]:
buildings = gpd.read_file("buildings.gpkg")

In [100]:
def fractal_dimension(Z, threshold=0):
    # only for 2d image
    assert(len(Z.shape) == 2)

    # from https://github.com/rougier/numpy-100 (#87)
    def boxcount(Z, k):
        S = np.add.reduceat(
            np.add.reduceat(Z, np.arange(0, Z.shape[0], k), axis=0),
                               np.arange(0, Z.shape[1], k), axis=1)

        # we count non-empty (0) and non-full boxes (k*k)
        return len(np.where((S > 0) & (S < k*k))[0])

    # transform Z into a binary array
    Z = (Z < threshold)

    # minimal dimension of image
    p = min(Z.shape)

    # greatest power of 2 less than or equal to p
    n = 2**np.floor(np.log(p)/np.log(2))

    # extract the exponent
    n = int(np.log(n)/np.log(2))

    # build successive box sizes (from 2**n down to 2**1)
    sizes = 2**np.arange(n, 1, -1)

    # actual box counting with decreasing size
    counts = []
    for size in sizes:
        counts.append(boxcount(Z, size))

    # fit the successive log(sizes) with log (counts)
    coeffs = np.polyfit(np.log(sizes), np.log(counts), 1)
    return -coeffs[0]

In [195]:
def get_street_image(city):
    
    street_widths = {'footway' : 0.5,
                     'steps' : 0.5,
                     'pedestrian' : 0.5,
                     'path' : 0.5,
                     'track' : 0.5,
                     'service' : 2,
                     'residential' : 3,
                     'primary' : 5,
                     'motorway' : 6}
    
    fig, ax = plt.subplots(1, 1, figsize=(20, 20), 
                           facecolor='k',
                           constrained_layout=True, 
                           subplot_kw=dict(aspect='equal'))
                     
    i = cities[cities['city']==city].index[0]
    
    node = nodes[nodes['city']==cities.city[i]]
    edge = edges[edges['city']==cities.city[i]]
    
    edge_linewidths = []
    
    default_width = 0.5
    
    for label in edge.highway:
        street_type = label[0] if isinstance(label, list) else label
    
        if street_type in street_widths:
            edge_linewidths.append(street_widths[street_type])
        else:
            edge_linewidths.append(default_width)

    edge.plot(ax=ax, color='w', lw=edge_linewidths)
    
    bbox = ox.utils_geo.bbox_from_point((cities['lat'][i], cities['lon'][i]), 800, project_utm=False)
    
    north, south, east, west = bbox
    ax.set_ylim((south, north))
    ax.set_xlim((west, east))

    ax.margins(0)
    ax.axis('off')

    fig.canvas.draw()
    img = np.frombuffer(fig.canvas.tostring_rgb(), dtype="uint8")
    img = img.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    
    plt.clf()
    
    return img[:, :, 0]

In [196]:
def get_building_image(city):
    
    fig, ax = plt.subplots(1, 1, figsize=(20, 20), 
                           facecolor='k',
                           constrained_layout=True, 
                           subplot_kw=dict(aspect='equal'))    
    
    i = cities[cities['city']==city].index[0]
    
    building = buildings[buildings['city']==cities.city[i]]
    
    building.plot(ax=ax, color='w')
    
    bbox = ox.utils_geo.bbox_from_point((cities['lat'][i], cities['lon'][i]), 800, project_utm=False)
    
    north, south, east, west = bbox
    ax.set_ylim((south, north))
    ax.set_xlim((west, east))

    ax.margins(0)
    ax.axis('off')

    fig.canvas.draw()
    img = np.frombuffer(fig.canvas.tostring_rgb(), dtype="uint8")
    img = img.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    
    plt.clf()
    
    return img[:, :, 0]

In [197]:
%%time
results = {}

for i in np.arange(0, len(cities.city)):
    
    city = cities.city[i]
    street_plot = get_street_image(city=city)
    fractal_streets = fractal_dimension(street_plot, threshold=0.25)
    
    building_plot = get_building_image(city=city)
    fractal_buildings = fractal_dimension(building_plot, threshold=0.25)
    
    results[city] = {'fractal_streets'  : fractal_streets,
                     'fractal_buildings': fractal_buildings}
    
    plt.close('all')

CPU times: user 38min 14s, sys: 31.1 s, total: 38min 45s
Wall time: 7min 8s


In [198]:
measures = pd.DataFrame(results).T.reset_index()
measures = measures.rename(columns={'index': "city"})

In [199]:
measures.to_csv("measures_frac.csv")

In [200]:
buildings = gpd.read_file("buildings.gpkg")

In [201]:
buildings = buildings.to_crs(3857)
buildings['area'] = buildings.geometry.area

In [202]:
measures = buildings.groupby('city').agg({'area': ['min', 'max', 'mean'],
                                          'osmid': 'count'}).reset_index()

In [203]:
measures.columns = measures.columns.map('_'.join)
measures = measures.rename(columns={'city_': 'city'})

In [204]:
measures.head()

,city,area_min,area_max,area_mean,osmid_count
0,Abilene,0.000000,13888.096798,1470.630733,313
1,Akron,7.376867,31455.214670,2361.457974,497
2,Albuquerque,0.000000,40821.136423,619.995841,1987
3,Alexandria,0.000000,9870.248877,256.829237,4933
4,Allen,13.428359,16796.669743,817.140416,994


In [205]:
measures.to_csv("measures_area.csv")